# モデルとしてLassoを指定したSelectFromModel関数
カテゴリ：ｗembedded method

## 中止　either the data is too noisy or the selection test too strict.

In [2]:
#import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random, math, warnings
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso
import pprint
import itertools

In [3]:
#valiable
num_select = 3

every_MCS = 100

In [4]:
b = [i / 1000 for i in range(1, 5000, 2)]
print("bのトップバッターは{}である".format(b[0])) #0を目指したかった
print("bのラストは{}である".format(b[-1])) #5を目指したかった
max_step = len(b)
print("max_stepは{}である".format(max_step)) 

bのトップバッターは0.001である
bのラストは4.999である
max_stepは2500である


In [5]:
X = pd.read_table('geneX_df.txt',sep ='\t')# 説明変数（こっちの特徴量上位5つ取り出す）
print("X.shape is {}".format(X.shape))

y = pd.read_table('ERY_df.txt' ,sep ='\t')# 目的変数
#print('type of y is {}'.format(type(y)))

num_sample = X.shape[0]#サンプル数
num_feature = X.shape[1]#特徴量の数

#Kをはじめにlist upするyの数とする

X.shape is (781, 16799)


In [6]:
len(y)

781

In [7]:
# 1 行目の 1 列目の値を取得
#df.iloc[1,1]

for i in range(len(y)):
    if y.iloc[i, 0]=="Positive":
        y.iloc[i, 0]=np.float(1)
    else:
        y.iloc[i, 0]=np.float(0)

series_y = y.loc[:, 'ER_Status_nature2012']

In [8]:
type(series_y)

pandas.core.series.Series

In [9]:
K = 2*210

In [13]:
# 正規化 (min-max スケーリング)
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler(feature_range=(-1, 1))
X = mms.fit_transform(X)

In [14]:
'''
clf = Lasso(alpha=3)
sfm = SelectFromModel(clf)
sfm.fit(X, y)
n_features = sfm.transform(X).shape[1]

print(sfm.get_support())

個々の段階でthe data is too noisy or the selection test too strictとエラーが出たので戻ってデータを正則化してみることに


clf = Lasso(alpha=3)
sfm = SelectFromModel(clf)
sfm.fit(X, y)
n_features = sfm.transform(X).shape[1]

print(sfm.get_support())

正則化してみたがエラーは変化せず
'''

[False False False ... False False False]


/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/sklearn/feature_selection/_base.py:81: UserWarning: No features were selected: either the data is too noisy or the selection test too strict.
  UserWarning)


In [ ]:
mask_ori

In [ ]:
#エネルギーが0(最低)となるのがいつか基準を先に計算しておきましょう
def E(y): #毎回のmaskとyが引数、合致度合いの計算を行う
    #selector = SelectKBest(score_func=chi2, k=num_select) #SelectKBest #chi2...分類
    forest = RandomForestClassifier()
    # define Boruta feature selection method
    select = boruta_py.BorutaPy(forest, n_estimators=10)
    select.fit(X, y)
    mask_each = select.support_
    
    energy_trait_sum = 1 #0を回避するため
    for i in range(len(mask_each)):
        if mask_each[i]!=mask_ori[i]:
            energy_trait_sum += 1
    return energy_trait_sum #もとのyのときのmask(これをmaskとしている)との合致度合い（大きい⇒よく合致している）

In [ ]:
E(series_y)

In [ ]:
series_y

👇ここで毎回```The kernel appears to have died. It will restart automatically.```
- num_sample = 10 できた
- num_sample = 100　もうだめ
- num_sample = 50　これでもだめ

### 今日教わったこと、わかったこと
```itertools.product```禁止

- そもそもモンテカルロはエネルギー差を見るもの。１つだけ変えるからエネルギー差がわかる

tried👇but didn't succeed

頻繁に起こるエラー
```'int' object is not subscriptable```。箇所は```ori_yj = y[j]```。```y```は```int```ではないのになぜ。

In [ ]:
series_y #一行前までseriesなのになぜか数字になる時がある。

In [ ]:
Ene = [] #これからi(温度)ごとに入れていく。一行に入っているのもlistにしたい(行i, 列k, 要素Ene)
y_list = [] #これからi(温度)ごとに、選択したyを記録していく。(行i, 要素num_tl)

Ene.append({})
y_list.append([])
for k in range(K):
    Ene[-1][k]= 1
    y_list[-1].append(series_y)
    
for i in range(1, max_step):   
    Ene.append({})
    y_list.append([])
    for k in range(K): #一度に熟すサンプル数
        #MCS
        #𝑃(B)/𝑃(A)>random ならば状態Bを採用し，そうでなければ状態Aを採用(Bを先、Aを過去とする)
        j = random.randrange(num_sample) #何番目の0 or 1をいじるか決める
        ori_yj = series_y[j]
        series_y[j] = 1-series_y[j]
        Ene[-1][k]= E(pd.Series(series_y))
        y_list[-1].append(series_y)
        
        #今回のターンはiね。
        r = math.exp(-Ene[i][k]*b[i]-(-Ene[i-1][k]*b[i-1])) #Ene[i-1][k]*b[i-1] - Ene[i][k]*b[i] #大きいほうが良い
        #rr = math.exp(-eene[0][0]*b[3]-(-eene[1][0]*b[2])) 
        R = np.random.uniform(0.0, 1.0)
        if R > r : #乱数R > エネルギー差の大きさを用いた計算
            Ene[-1][k] = Ene[i-1][k]
            y_list[-1][k] = y_list[i-1][k]
            series_y = y_list[i-1][k]
    
    
    #Eneのi行目に数値が全部入って、かつ👇を満たすようだったら
    if i%every_MCS == 0:
        Ei_sorted = sorted(Ene[-1].items(), key=lambda x:x[1]) #辞書をvalueを元にsort
        tuple_kE_list = Ei_sorted[0:int(len(Ei_sorted))] #tuple(k, Ene)
        
        #y_list[-1](y_listの最後尾)を並べ替え
        y_list_sort = [y_list[-1][tuple_kE_list[k][0]] for k in range(K)]
        
        #ここまでで、Ei_sorted, tuple_kE_list, y_list_sortによってsortedされたEneとy_listが得られた
        
        #Ene[i][k]はいつでも0以外にしないとエラーが起こる
        sumQ = sum([math.exp(-(b[i]-b[i-1])*Ene[i][k]) for k in range(K)])
        
        #すべてのアイテムについてコピー可能数を計算する
        tbi = [K*math.exp(-(b[i]-b[i-1])*tuple_kE_list[k][1])/sumQ for k in range(K)]
        
        next_K = 0 #次のKの値を決めよう
        reserve_min_num = 0
        for k in range(K):
            if next_K <= K:
                if tbi[k] % 1 >= 0.5:
                    tbi[k] = tbi[k] // 1 +1
                else:
                    tbi[k] = tbi[k] // 1
                    
                next_K += tbi[k]
            else:
                reserve_min_num = k-1
                break #forループを抜ける
        
        
        #k, k+1, ..., Kにおいてtuple_kE_list[k][1]を0, 1, ...k-1のととって変える
        for k in range(reserve_min_num): #次にも引き継がれるk
            for kk in range(tbi[k]): #kによって含まれる要素数が異なる
                ll = tbi[k]*k + kk + 1 #エネルギー大きいほうからll番目
                ori_k = tuple_kE_list[-ll][0]#いわゆるkを知る
                y_list[-1][ori_k] = y_list_sort[-ll]
                Ene[-1][ori_k] = tuple_kE_list[-ll][1]
                
        K = int(next_K)
                

if len(Ene) == max_step:               
    #初期値を抜く
    del Ene[0]
    del y_list[0]

In [ ]:
pd.DataFrame(Ene)

In [ ]:
Ene_v_sum_list = []
Ene_v_sum_list_append = Ene_v_sum_list.append
for each_Ene in Ene:
    Ene_v_sum = sum([Ene_v for Ene_v in each_Ene.values()])
    Ene_v_sum_list_append(Ene_v_sum/K)
#Ene_v_sum_list
plt.plot(Ene_v_sum_list)
plt.savefig('meeting_0428_0_figure_0.png')
print(Ene_v_sum_list[-1])

In [ ]:
#毎度(約)K個あるアイテムの内、0, 100, 200, 400番目のアイテムのエネルギーの移り変わりをplotする
ene0 = []
ene100 = []
ene200 = []
ene400 = []
for i in range(len(Ene)):
    ene0.append(Ene[i][0])
    ene100.append(Ene[i][100])
    ene200.append(Ene[i][200])
    ene400.append(Ene[i][400])

plt.plot(ene0)
plt.plot(ene100)
plt.plot(ene200)
plt.plot(ene400)
plt.savefig('meeting_0428_0_figure_1.png')

In [ ]:
#エネルギーが最小(1)のアイテム数の移り変わりをplotする

ene0_num = []
for i in range(len(Ene)):
    ene0_num.append(len([k for k, v in Ene[i].items() if v == 1]))
    
plt.plot(ene0_num)
plt.savefig('meeting_0428_0_figure_2.png')